In [65]:
from lxml import html
import requests
from bs4 import BeautifulSoup
import urllib
import urllib2
import pandas as pd
import numpy as np

In [66]:
r = urllib.urlopen('http://www.buda.org/leagues/past-leagues')
soup = BeautifulSoup(r, 'html.parser')

In [67]:
iframe = soup.find_all('iframe')[0]
response = urllib2.urlopen(iframe.attrs['src'])
iframe_soup = BeautifulSoup(response)

In [68]:
leaguelinks = [i.a['href'] for i in iframe_soup.find_all("td", class_="infobody")]

In [ ]:
# define the dictionary that will contain all player ratings
all_players = {}

# loop over all leagues in the BUDA database
for link in leaguelinks:

    # extract the league id for this league
    leagueid = link[link.index('league=') + 7:]

    # scrape the scores for this league
    leaguescoreurl = 'http://www.buda.org/hatleagues/scores.php?section=showLeagueSchedule&league=' + leagueid + '&byDivision=1&showGames=0'
    response = urllib2.urlopen(leaguescoreurl)
    leaguescore_soup = BeautifulSoup(response)

    # assemble the data of team ratings for this league
    data = []
    try:
        table = leaguescore_soup.find_all('table', attrs={'class':'info'})[1]
    except IndexError:
        print("Unable to find a database of scores for league {}".format(leagueid))
        continue
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('th')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values

    # convert to dataframe and drop irrelevant columns
    dfdata = pd.DataFrame(data)
#     print(leagueid, dfdata.columns)
    dfdata.columns = dfdata.ix[0, :]#['Team', 'Record', 'Plus/Minus', 'Tourney Qualifying games']
#     print(leagueid, dfdata.columns)
    dfdata = dfdata.drop(0).reset_index()

    # establish base ratings by division
    divnames = ['4/3 Div 1', '4/3 Div 2', '4/3 Div 3', 
                '5/2 Div 1', '5/2 Div 2', '5/2 Div 3', 
                'Open Div 1', 'Open Div 2']
    divbases = [1800, 1400, 1000, 1600, 1200, 900, 1400, 1200]
    dfdata['div'] = np.zeros(len(dfdata))
    for i in range(len(divnames)-1):
        try:
            divstart = np.where(dfdata['Team'] == divnames[i])[0][0]
        except IndexError:
            print("{} not found, skipping league {}".format(divnames[i], leagueid))
            continue
        try:
            divend = np.where(dfdata['Team'] == divnames[i + 1])[0][0]
        except IndexError:
            print("{} not found, skipping league {}".format(divnames[i + 1], leagueid))
            continue
        dfdata.ix[divstart + 1: divend, 'div'] = divbases[i]
    dfdata.ix[divend + 1:, 'div'] = divbases[-1]

    # remove the division dividers from the dataframe
    for i in range(len(divnames)):
        dfdata = dfdata.drop(dfdata.index[dfdata['Team'] == divnames[i]])

    # generate the average goal differential column
    dfdata['wins'] = dfdata['Record'].apply(lambda x: int(x.split('-')[0]))
    dfdata['losses'] = dfdata['Record'].apply(lambda x: int(x.split('-')[1]))
    dfdata['games'] = dfdata['wins'] + dfdata['losses']
    dfdata['avgplusminus'] = dfdata['Plus/Minus'].astype('float') / dfdata['games']

    # assert that an average goal differential per game of +5 gives +300 rating points.
    dfdata['rating'] = dfdata['div'] + 60. * dfdata['avgplusminus']

    # scrape the list of teams for this league
    teamsurl = 'http://www.buda.org/hatleagues/rosters.php?section=showTeams&league=' + leagueid
    response = urllib2.urlopen(teamsurl)
    teams_soup = BeautifulSoup(response)

    # generate list of team ids and names for this league
    tdlist = teams_soup.find_all('td', class_='infobody')
    teamids = []
    teamnames = []
    for i in tdlist:
        try:
            url = i.a['href']
            idindex = url.index('team=')
            whichindex = url.index('which=')
            teamids.append(url[idindex+5:whichindex-1])
            teamnames.append(i.a.get_text())
        except:
            continue

    # find all players associated with each team
    # link the team rating to each player on that team
    for teamid, teamname in zip(teamids, teamnames):
        teamrating = dfdata.ix[dfdata['Team'] == teamname.strip(' '), 'rating'].values[0]

        teamurl = 'http://www.buda.org/hatleagues/rosters.php?section=showTeamRoster&team=' + teamid
        response = urllib2.urlopen(teamurl)
        roster_soup = BeautifulSoup(response)

        players = [i.get_text() for i in roster_soup.find_all("td", class_="infobody")]
        for player in players:
            if player == '':
                print(teamname)
            if player in all_players:
                all_players[player].append(teamrating)
            else:
                all_players[player] = [teamrating]
    print("Finished successfully with leage {}".format(leagueid))

Unable to find a database of scores for league 40491
Unable to find a database of scores for league 40278
Unable to find a database of scores for league 40273
Unable to find a database of scores for league 40268
Unable to find a database of scores for league 40264
Unable to find a database of scores for league 40258
Unable to find a database of scores for league 40253
Unable to find a database of scores for league 40249
Unable to find a database of scores for league 40245
4/3 Div 1 not found, skipping league 39633
Unable to find a database of scores for league 39960
Unable to find a database of scores for league 39939
Unable to find a database of scores for league 39904
Unable to find a database of scores for league 39678
Unable to find a database of scores for league 39673
4/3 Div 3 not found, skipping league 39641
4/3 Div 1 not found, skipping league 39628
4/3 Div 1 not found, skipping league 39622
4/3 Div 1 not found, skipping league 39616
Unable to find a database of scores for lea

AttributeError: 'NoneType' object has no attribute 'split'

> <ipython-input-78-977e69ca6509>(56)<lambda>()
     55     # generate the average goal differential column
---> 56     dfdata['wins'] = dfdata['Record'].apply(lambda x: int(x.split('-')[0]))
     57     dfdata['losses'] = dfdata['Record'].apply(lambda x: int(x.split('-')[1]))

ipdb> dfdata
0   index                           Team   Record Plus/Minus  \
1       2                    GrassBurner   17-5-0         89   
2       3                     Zerg Rush!   13-1-0         66   
3       4                      Flowchart   13-6-0         48   
4       5                      GoThrilla   12-3-0         43   
5       6                       Swingers    9-6-0         40   
6       7              Capture the Dream    8-4-0         26   
7       8                     Blocktopus    6-3-0         27   
8       9                            BBN   1-10-0        -43   
9      10                        Booosh!    3-9-1        -65   
11     12                     Turtle Boy   12-4-1         45   
12  

In [62]:
all_players

{u'': [1813.3333333333333,
  1868.5714285714287,
  2005.7142857142858,
  2061.1764705882351,
  1812.0,
  1860.0,
  1687.5,
  1626.0,
  2070.0,
  1202.0,
  1516.25,
  1657.1428571428571,
  1463.3333333333333,
  1526.3157894736842,
  1262.0,
  1373.3333333333333,
  1271.4285714285716,
  1386.1538461538462,
  1164.2857142857142,
  1318.8235294117646,
  1538.0,
  1460.0,
  1430.0,
  1486.4000000000001,
  946.66666666666674,
  1445.8823529411766,
  1490.0,
  670.0,
  929.09090909090912,
  1123.3333333333333,
  718.0,
  426.66666666666663,
  922.5,
  808.0,
  1032.7272727272727,
  1133.8461538461538,
  1480.0,
  1940.0,
  2028.5714285714287,
  1904.0,
  1589.090909090909,
  1342.5,
  1326.6666666666667,
  900.0,
  1086.6666666666667,
  1234.2857142857142,
  1088.5714285714287,
  1133.3333333333333,
  1265.0,
  893.33333333333337,
  1180.0,
  1320.0,
  1435.7142857142858,
  1165.0,
  1509.2307692307693,
  982.5,
  960.0,
  1065.0,
  1008.5714285714286,
  1020.0,
  652.5,
  973.84615384615381,